# Sugestão de cidade do Estado do Rio Grande do Sul pelo Índice de Criminalidade
### Para o trabalho final da disciplina de Sistemas Operacionais foi escolhido os bancos de dados Índice de Criminalidade (RS, 2023) e Estimativas Populacionais (RS, 2022). Os dados são reais e estão disponíveis no portal Dados Abertos RS do Estado do Rio Grande do Sul.
###  Este trabalho tem como objetivo análisar a similaridade, usando a métrica de distância eucliana, das cidades e os ratigns dos dos usuários e, então responder: "Quais as cidades o algoritmo sugere?".
Indíce de Criminalidade no RS: [https://dados.rs.gov.br/dataset/indicadores-criminais-de-2023](https://dados.rs.gov.br/dataset/indicadores-criminais-de-2023)

Estimativas Populacionais no RS: [https://dados.rs.gov.br/dataset/dee-4259/resource/ce259dd9-c479-4a18-90b3-40098e6deb26](https://dados.rs.gov.br/dataset/dee-4259/resource/ce259dd9-c479-4a18-90b3-40098e6deb26)

## Pré-processamento
Houve a necessidade de realizar o pré-processamento dos dados, pois foram disponibilizados em planilhas Excel com formatações diversas.

In [274]:
import pandas as pd

Abertura dos arquivos de criminalidade por cidade em cada mês.

In [275]:
# Lendo os arquivos csv e concatenando em um único dataframe
meses = ['janeiro', 'fevereiro', 'marco', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

dfs = []

for mes in meses:
        arquivo = pd.read_csv(f'data\\tocsv\\{mes}.csv', delimiter=';')
        dfs.append(arquivo)

df = pd.concat(dfs, ignore_index=True)

df_criminal = df.groupby(['municipios', 'ibge']).sum().reset_index()


df.to_csv('data\\tocsv\\total.csv', sep=';', index=False)

df_criminal.head()


,municipios,ibge,homicidio_doloso,total_vitimas_homicidio_doloso,latrocinio,furtos,abigeato,furto_veiculo,roubos,roubo_veiculo,estelionato,delitos_armas_municoes,entorpecente_posse,entorpecente_trafico,vitimas_latrocinio,vitimas_lesao_corporal_morte,total_vitimas_crimes_violentos
0,acegua,4300034,0,0,0,20,8,0,3,0,27,4,3,0,0,0,0
1,agua santa,4300059,1,1,0,20,3,4,1,0,13,2,0,0,0,0,2
2,agudo,4300109,1,1,0,102,2,2,3,1,51,3,7,10,0,0,1
3,ajuricaba,4300208,0,0,0,26,1,0,1,0,28,2,3,0,0,0,0
4,alecrim,4300307,1,1,0,36,10,0,2,0,17,10,0,4,0,0,1


Abertura do arquivo de quantidade de população por cidade em cada ano.

In [276]:
# Lendo o arquivo de população
df_pop = pd.read_csv('data\\tocsv\\populacao.csv', delimiter=';')

# Substituindo os valores de '.' e '-' por 0 e convertendo para inteiro
for coluna in df_pop.columns[4:]:
    df_pop[coluna] = df_pop[coluna].str.replace('.', '').str.replace('-', '0').astype(int)

df_pop.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   municipios  497 non-null    object 
 1   ibge        497 non-null    int64  
 2   latitude    497 non-null    float64
 3   longitude   497 non-null    float64
 4   2010        497 non-null    int32  
 5   2011        497 non-null    int32  
 6   2012        497 non-null    int32  
 7   2013        497 non-null    int32  
 8   2014        497 non-null    int32  
 9   2015        497 non-null    int32  
 10  2016        497 non-null    int32  
 11  2017        497 non-null    int32  
 12  2018        497 non-null    int32  
 13  2019        497 non-null    int32  
 14  2020        497 non-null    int32  
 15  2021        497 non-null    int32  
dtypes: float64(2), int32(12), int64(1), object(1)
memory usage: 39.0+ KB


União dos dois banco de dados.

In [277]:
# O merge é feito com base na coluna ibge como forma de chave estrangeira com o método inner
df_merged = pd.merge(df_criminal, df_pop, how = 'inner', on = 'ibge').drop(columns=['municipios_y'])
df_merged.rename(columns={'municipios_x': 'municipios'}, inplace=True)

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   municipios                      497 non-null    object 
 1   ibge                            497 non-null    int64  
 2   homicidio_doloso                497 non-null    int64  
 3   total_vitimas_homicidio_doloso  497 non-null    int64  
 4   latrocinio                      497 non-null    int64  
 5   furtos                          497 non-null    int64  
 6   abigeato                        497 non-null    int64  
 7   furto_veiculo                   497 non-null    int64  
 8   roubos                          497 non-null    int64  
 9   roubo_veiculo                   497 non-null    int64  
 10  estelionato                     497 non-null    int64  
 11  delitos_armas_municoes          497 non-null    int64  
 12  entorpecente_posse              497 

Normalização dos dados.

In [278]:
from sklearn.preprocessing import StandardScaler

In [279]:
# Lista das colunas extras que não são necessárias
colunas_extras = df_merged.columns[16:30]

# Removendo as colunas extras
df = df_merged.drop(colunas_extras, axis=1)

df.set_index('municipios', inplace=True)

# Salvando o dataframe em um arquivo csv
df.to_csv('data\\tocsv\\dados_pop_criminalidade.csv', sep=';', index=False)

df.head()

df.shape



(497, 16)

In [280]:
# Removendo as colunas de taxa total pois são redundantes
df.drop(['total_vitimas_homicidio_doloso'], axis='columns', inplace=True)
df.drop(['latrocinio'], axis='columns', inplace=True)

# Selecionando somente as colunas do tipo float
df.drop(['ibge'], axis='columns', inplace=True)

df.rename(columns={'2021': 'populacao'}, inplace=True)

In [281]:
# Normalizando os dados média 0 e desvio padrão 1
scaler = StandardScaler(with_std=True, with_mean=True)
data_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

data_scaled.head()

,homicidio_doloso,furtos,abigeato,furto_veiculo,roubos,roubo_veiculo,estelionato,delitos_armas_municoes,entorpecente_posse,entorpecente_trafico,vitimas_latrocinio,vitimas_lesao_corporal_morte,populacao
municipios,,,,,,,,,,,,,
acegua,-0.240983,-0.165023,-0.016371,-0.189482,-0.091215,-0.108250,-0.174139,-0.221637,-0.264673,-0.205293,-0.202622,-0.193052,-0.234986
agua santa,-0.163812,-0.165023,-0.347109,-0.145451,-0.094148,-0.108250,-0.190466,-0.270616,-0.287715,-0.205293,-0.202622,-0.193052,-0.238138
agudo,-0.163812,-0.106419,-0.413257,-0.167466,-0.091215,-0.093268,-0.146148,-0.246127,-0.233949,-0.141324,-0.202622,-0.193052,-0.081756
ajuricaba,-0.240983,-0.160734,-0.479404,-0.189482,-0.094148,-0.108250,-0.172972,-0.270616,-0.264673,-0.205293,-0.202622,-0.193052,-0.197751
alecrim,-0.163812,-0.153588,0.115925,-0.189482,-0.092681,-0.108250,-0.185801,-0.074700,-0.287715,-0.179705,-0.202622,-0.193052,-0.210964


## PCA - Redução de Dimensionalidade
### O dataset agora tem dados suficientes para ranquear as cidades em mais seguras e não seguras com dados proporcionais a população. Entretanto, existem muitas colunas. A alta dimensionalidade deixará o processo de aprendizado confuso criando um sobreajuste nos dados. Então é aplicado o PCA, uma técnida de redução de dimensionalidade.
### Para fins de demonstração, será usado somente uma amostra de 20 cidades.



In [282]:
# Selecionando 20 linhas aleatórias
df_sample= data_scaled.sample(n=25, axis=0, random_state=999)

df_sample.index



Index(['hulha negra', 'sao valentim do sul', 'xangri-la', 'nova padua',
       'catuipe', 'esmeralda', 'lajeado do bugre', 'cruz alta', 'nova ramada',
       'jaguarao', 'nova petropolis', 'charqueadas', 'torres', 'sarandi',
       'fortaleza dos valos', 'almirante tamandare do sul', 'esteio',
       'frederico westphalen', 'canela', 'piratini', 'vale verde',
       'restinga seca', 'santo antonio da patrulha', 'passo do sobrado',
       'sao sepe'],
      dtype='object', name='municipios')

In [283]:
import numpy as np

from sklearn.decomposition import PCA


In [284]:
# Aplicando a PCA
pca = PCA()
components = pca.fit_transform(df_sample)

components.shape[1]

13

In [285]:
# Analise de como os componentes estão explicando os dados com a variância explicada
pca.explained_variance_ratio_

array([3.81403134e-01, 3.35609185e-01, 1.40727454e-01, 7.49776034e-02,
       4.64638856e-02, 1.41957814e-02, 2.98215223e-03, 2.68379209e-03,
       5.28358386e-04, 3.23701599e-04, 6.66889179e-05, 3.82634109e-05,
       0.00000000e+00])

In [286]:
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go

import matplotlib.pyplot as plt
%matplotlib inline

In [287]:
# Plotando o gráfico da variância explicada acumulada
px.area(
    x=range(1, pca.explained_variance_ratio_.cumsum().shape[0] + 1),
    y=pca.explained_variance_ratio_.cumsum(),
    labels={"x": "Componentes Principais", "y": "Variância Explicada Acumulada"}
)


In [288]:
n = 4

# Aplicando a PCA com n componentes
pca = PCA(n_components=n)
components = pca.fit_transform(df_sample)

components.shape[1]

4

Dataframe agora utilizando os componentes principais para explicar os dados de criminalidade nas cidades.

In [289]:
df_pca = pd.DataFrame(data = components, columns = ['PC1', 'PC2', 'PC3', 'PC4'])
df_pca['municipios'] = df_sample.index
df_pca.set_index('municipios', inplace=True)
df_pca.head()

,PC1,PC2,PC3,PC4
municipios,,,,
hulha negra,-0.601557,0.075443,-0.145768,0.006247
sao valentim do sul,-0.682314,-0.512254,-0.072228,0.019699
xangri-la,0.044404,-0.434083,-0.035557,-0.017995
nova padua,-0.710398,-0.512442,-0.073661,0.017818
catuipe,-0.479446,-0.042584,-0.122929,0.173788


In [290]:
total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_matrix(
    df_pca,
    title=f'Taxa Total de Variancia Explicada por 4 PC: {total_var:.2f}%',
    dimensions=df_pca.columns,
    color=df_pca.index,
)
fig.update_traces(diagonal_visible=False)
fig.show()

""" Foi utilizado a biblioteca Plotly para a visualização dos dados. Documentação do gráfico PCA: https://plotly.com/python/pca-visualization/"""

' Foi utilizado a biblioteca Plotly para a visualização dos dados. Documentação do gráfico PCA: https://plotly.com/python/pca-visualization/'

In [291]:
total_var_2 = pca.explained_variance_ratio_[:2].sum() * 100
total_var_2

# Calculando as cargas dos componentes
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# Plotando o gráfico de dispersão com os componentes principais
fig = px.scatter(
    components, x=0, y=1, color=df_pca.index,
    labels={'0': 'PC1', '1': 'PC2', 'color': 'Município'},
    title=f'Taxa Total de Variancia Explicada por 2 PC: {total_var_2:.2f}%')

features = df_sample.columns

# Adicionando os vetores de carga ao gráfico
for i, feature in enumerate(features):
    fig.add_annotation(
        ax=0, ay=0,
        axref="x", ayref="y",
        x=loadings[i, 0],
        y=loadings[i, 1],
        showarrow=True,
        arrowsize=2,
        arrowhead=2,
        xanchor="right",
        yanchor="top"
    )
    
    fig.add_annotation(
        x=loadings[i, 0],
        y=loadings[i, 1],
        ax=0, ay=0,
        xanchor="center",
        yanchor="bottom",
        text=feature,
        yshift=5,
    )
# Invertendo o eixo y para que o gráfico fique na posição correta    
fig.update_yaxes(autorange="reversed")
fig.show()

""" Foi utilizado a biblioteca Plotly para a visualização dos dados. Documentação do gráfico PCA: https://plotly.com/python/pca-visualization/"""

' Foi utilizado a biblioteca Plotly para a visualização dos dados. Documentação do gráfico PCA: https://plotly.com/python/pca-visualization/'

In [292]:
""" i, j = 0, 1 # which components
scale_arrow = s_ = 2
components[:,1] *= -1
pca.components_[1] *= -1 # flip the y-axis
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.scatter(components[:,0], components[:,1])
ax.set_xlabel('PC%d' % (i+1))
ax.set_ylabel('PC%d' % (j+1))
for k in range(pca.components_.shape[1]):
    ax.arrow(0, 0, s_*pca.components_[i,k], s_*pca.components_[j,k])
    ax.text(s_*pca.components_[i,k],
            s_*pca.components_[j,k],
            df_sample.columns[k]) """

" i, j = 0, 1 # which components\nscale_arrow = s_ = 2\ncomponents[:,1] *= -1\npca.components_[1] *= -1 # flip the y-axis\nfig, ax = plt.subplots(1, 1, figsize=(8, 8))\nax.scatter(components[:,0], components[:,1])\nax.set_xlabel('PC%d' % (i+1))\nax.set_ylabel('PC%d' % (j+1))\nfor k in range(pca.components_.shape[1]):\n    ax.arrow(0, 0, s_*pca.components_[i,k], s_*pca.components_[j,k])\n    ax.text(s_*pca.components_[i,k],\n            s_*pca.components_[j,k],\n            df_sample.columns[k]) "

## Sistema de Recomendação


In [310]:
df_recom = df_pca.drop(columns=['cluster'])

df_recom

,PC1,PC2,PC3,PC4
municipios,,,,
hulha negra,-0.601557,0.075443,-0.145768,0.006247
sao valentim do sul,-0.682314,-0.512254,-0.072228,0.019699
xangri-la,0.044404,-0.434083,-0.035557,-0.017995
nova padua,-0.710398,-0.512442,-0.073661,0.017818
catuipe,-0.479446,-0.042584,-0.122929,0.173788
esmeralda,-0.586841,-0.436212,-0.077948,0.159723
lajeado do bugre,-0.681381,-0.509419,-0.073198,0.045610
cruz alta,1.669484,0.259451,-0.038734,1.027021
nova ramada,-0.680811,-0.443985,-0.081915,0.039932


In [311]:
from sklearn.metrics.pairwise import euclidean_distances

In [312]:
# Calculando a distância euclidiana entre os componentes principais
dist = euclidean_distances(df_recom)
dist.shape


(25, 25)

In [313]:
df_recom = df_recom.reset_index()
indices = pd.Series(df_recom.index, index=df_recom['municipios'])
indices

municipios
hulha negra                    0
sao valentim do sul            1
xangri-la                      2
nova padua                     3
catuipe                        4
esmeralda                      5
lajeado do bugre               6
cruz alta                      7
nova ramada                    8
jaguarao                       9
nova petropolis               10
charqueadas                   11
torres                        12
sarandi                       13
fortaleza dos valos           14
almirante tamandare do sul    15
esteio                        16
frederico westphalen          17
canela                        18
piratini                      19
vale verde                    20
restinga seca                 21
santo antonio da patrulha     22
passo do sobrado              23
sao sepe                      24
dtype: int64

In [314]:
df_ratings = pd.read_csv('data\\tocsv\\ratings.csv', delimiter=',')

df_ratings.set_index('first_name', inplace=True)

for i in range (0, len(df_sample)):
    df_ratings.rename(columns={f'rating_{i}': df_sample.index[i]}, inplace=True)

df_ratings = df_ratings.T

#somar os ratings de cada municipio
df_ratings['rating_mean'] = df_ratings.mean(axis=1, numeric_only=True)



df_ratings.head()

first_name,Angelita,Leland,Bartholomeus,Desdemona,Costanza,Christan,Malinde,Nikolaus,Myriam,Anallese,...,Lindie,Yalonda,Korie,Ronnie,Idelle,Jacquie,Nickolas,Niko,Bill,rating_mean
hulha negra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sao valentim do sul,NaN,1.8,NaN,1.9,2.1,1.1,4.7,NaN,4.4,4.0,...,NaN,NaN,4.5,3.2,NaN,4.6,NaN,1.0,NaN,3.060000
xangri-la,NaN,1.9,NaN,NaN,4.8,1.0,2.1,4.5,4.0,4.4,...,NaN,1.0,1.2,2.4,1.6,2.3,4.0,4.6,1.6,2.461538
nova padua,NaN,1.8,3.7,4.7,1.2,NaN,NaN,1.6,NaN,NaN,...,4.8,3.7,2.7,1.3,3.6,NaN,2.1,NaN,4.0,2.797222
catuipe,NaN,NaN,NaN,1.8,NaN,NaN,NaN,4.2,NaN,NaN,...,3.4,NaN,NaN,NaN,NaN,NaN,NaN,2.9,NaN,2.733333


In [315]:
def score_similar(user, indices, dist, num_recommend):

  if len(user['municipios']) == 0:

    # Recomendar uma lista de itens para um usuário novo (sem ratings)
    if len(user['ratings']) == 0:

      # Recomendar itens mais populares entre todos os usuários
      score = df_ratings['rating_mean'].sort_values(ascending=False)

      return indices[score[0:num_recommend].index]

    elif len(user['ratings']) < 0:
      return 'Erro, ranting sem municípios e negativo.'
    else:
      return 'Erro, ranting sem município.'

  elif len(user['municipios']) > 0:

    # Recomendar uma lista de itens para um usuário antigo de acordo com os próprios ratings
    if len(user['ratings']) > 0:
      
      i=indices[user['municipios']]

      res = pd.DataFrame(dist[i])
      res = res.drop(i, axis='columns').T

      # Realiza a multiplicação de matrizes entre res e os ratings do usuário, resultando em um vetor de scores.
      score = res.dot(user['ratings']).sort_values(ascending=False)
      

      return indices[score[0:num_recommend].index]
    
    # Erro se o usuário tiver ratings negativos
    elif len(user['ratings']) < 0:
      return 'Erro, rating negativo.'
    
    # Sugerir avaliar os municipios visitados
    elif len(user['ratings']) == 0:
      return 'Que tal avaliar os municípios visitados?'

  else:
    return 'Erro.'
  


In [322]:
n_rec = 5

usr1 = {
  'municipios': ['canela','xangri-la','cruz alta'],
  'ratings': [5,2,1]
}

usr2 = {
  "municipios": [],
  "ratings": []
}

usr3 = {
  "municipios": ['canela','santo antonio da patrulha','cruz alta'],
  "ratings": [1,2.9,5]
}

In [ ]:
print(score_similar(usr1, indices, dist, n_rec))

In [ ]:
print(score_similar(usr2, indices, dist, n_rec))

In [ ]:
print(score_similar(usr3, indices, dist, n_rec))